In [ ]:
from glob import glob
import pint.models as model
# pip install git+https://github.com/telegraphic/pygdsm
# or install from source (git clone https://github.com/telegraphic/pygdsm)
import pygdsm
import os
import numpy as np
#%matplotlib inline
import matplotlib.pyplot as plt
from astropy import units as u, constants as c

In [ ]:
def get_prof_nbin(fname):
    x,y,z,intensity = np.loadtxt(fname,unpack=True,dtype='float')
    nbin = len(intensity)
    
    return intensity,nbin

def get_on_binwidth(folded_profile,threshold = 0.925,interpulse=False):
    
    box_size = []
    max_conv = []
    nbin = len(folded_profile)
    for jj in range(1,nbin):
        box = np.zeros(nbin)
        box_width = jj
        box_size.append(box_width)
        box[0:box_width] += 1.0
        conv = np.convolve(box,folded_profile)
        max_conv.append(np.max(conv))
    
    on_width = np.argmin((np.array(max_conv)-threshold)**2) + 1
    return on_width

def roll_prof(profile,max_phase=0.3):
    nbin = len(profile)
    max_ind = np.argmax(profile)
    shift = int(max_phase * nbin) - max_ind
    return np.roll(profile,shift)
    
def get_si(freq1,flux1,freq2,flux2):
    return np.log10(flux2/flux1)/np.log10(freq2/freq1)

class OnOff:
    """
    This class contains the functionality to parse .onoff files;
    the corresponding object contains relevant information.
    """
    def __init__(self, filename):
        """
        Initialization method.

        Parses and stores header values, profile intensities, and on/off info.

        Parameters
        ==========
        filename (str) : path to the .onoff file
        """
        self.filename = filename
        
        with open(self.filename,'r') as f:
            onoff_contents = f.readlines()
            
        for line in onoff_contents:

            # Parse header to get relevant profile info
            if line.startswith("#"):
                for item in line.split():
                    if "tobs" in item:
                        self.tobs = float(item.split("=")[1].replace(",",""))
                    elif "snr" in item:
                        self.snr = float(item.split("=")[1].replace(",",""))
                    elif "duty" in item:
                        self.duty = float(item.split("=")[1].replace(",",""))
                    elif "degradation" in item:
                        self.deg = float(item.split("=")[1].replace(",",""))
                    elif "file" in item:
                        self.prof_file = item.split("=")[1]
                    else:
                        continue
            else:
                continue
            
        self.profile, self.onoff = np.loadtxt(self.filename,dtype='float',unpack=True)
        self.nbin = len(self.profile)
        self.wid = int(self.nbin * self.duty) # (integer) box_width

In [ ]:
prof350 = np.sort(glob("data/*350MHz*profile"))
tobs350 = np.array([3309.8,5906.3,2583.0,14112.3,4015.8,10225.4,
                    9557.0,4539.9,3935.3,8350.4,6534.4,7637.0])
deg350  = np.array([0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.86,0.9,0.9,0.9,0.9])
prof820 = np.sort(glob("data/*820MHz*profile"))
tobs820 = np.array([584.01,5480.4,3686.3,4944.2,1188.0,5841.1,4309.6,
                    564.05,564.05,3888.0,4068.6])
deg820  = np.array([0.9,0.8,0.9,0.9,0.5,0.9,0.9,0.8,0.9,0.9,0.9])
print(f"Number of 350/820 profiles: {len(prof350)}/{len(prof820)}")

In [ ]:
"""The convolution makes boxes match profiles better for plotting, but
this is an iterative/by-eye process of finding box widths that match
10% pulse peak (W10). Change index/box_width until the plot looks good
and a .onoff file will be written with useful info in the header and 1/0
denoting bins where the pulsar signal is on/off.

Observation times (tobs) listed here come from `vap -c length` on the original
archives and degradation is based on position offset during observing. I
generated profiles for the most part only using scans where the position offset
caused < 10% degradation (deg=0.9), but in some cases that was not possible. See
notes from 10/21/2022."""

OVERWRITE_ONOFF = False

index = 7
#tobs = tobs350[index]
tobs = tobs820[index]
#ff = prof350[index]
ff = prof820[index]
#degradation = deg350[index]
degradation = deg820[index]

# check if .onoff file already exists, use it 
of = ff.replace('profile','onoff')
if os.path.exists(of):
    print(f" ON/OFF file exists: {of}")
    oo_obj = OnOff(of)
    print(f"Width (from .onoff): {oo_obj.wid}")
    print(f"Degr. (from .onoff): {oo_obj.deg}")
    print(f"  SNR (from .onoff): {oo_obj.snr}")
    profile, on_window, nbin = oo_obj.profile, oo_obj.onoff, oo_obj.nbin
    on_window *= np.max(profile)
    print()
    
if OVERWRITE_ONOFF:
    print(f"File: {ff}")
    prof,nbin = get_prof_nbin(ff)
    normed_prof = prof/np.sum(prof)
    profile = roll_prof(normed_prof,max_phase=0.5)

    box_width = 40
    box = np.zeros(nbin)
    box[0:box_width] += max(profile)

    if ('J1221' in ff) and ('350MHz' in ff):
        box[91:102] += 0.5*max(profile) # Add for J1221, 350 MHz
    if ('J1221' in ff) and ('820MHz' in ff):
        box[31:55] += 0.1*max(profile) # Add for J1221, 820 MHz
        box[91:98] += 0.5*max(profile) # Add for J1221, 820 MHz

    conv = np.convolve(box,profile)
    conv_box = np.roll(box,nbin-box_width)
    on_window = np.roll(conv_box,np.argmax(conv))
    off_inds = np.where(on_window == 0.0)

    box_width = len(np.where(box != 0.0)[0])
    print(f"Width: {box_width}")

    # Calculate S/N
    off_mean = np.mean(profile[off_inds])
    off_std = np.std(profile[off_inds])
    snr = np.sum((profile-off_mean)/(off_std*np.sqrt(box_width)))
    print(f"SNR: {snr}")
    
plt.plot(on_window)
plt.plot(profile)
plt.plot(np.zeros(nbin)+0.1*np.max(profile),ls='--',color='gray')

# WRITE ROLLED PROFILE + CORRESPONDING ON/OFF INFO TO FILE (.onoff)
if OVERWRITE_ONOFF:
    onoff = [np.ceil(x) for x in on_window]
    with open(of,'w') as f:
        file_only = ff.split("/")[1]
        duty=box_width/nbin
        #write header
        f.write(f"# file={file_only} \n")
        f.write(f"# tobs={tobs}, snr={snr:.5f}, duty={duty:.5f}, degradation={degradation} \n")
        for pp,oo in zip(profile,onoff):
            f.write(f"{pp:.8f} {oo}\n")

In [ ]:
gsm = pygdsm.GlobalSkyModel()

beta = 1.3
Gain = 2.0
Npol = 2

FluxDict = {}
par_paths = np.sort(glob("data/*[0-9]_swiggum+22.par"))
for pp in par_paths:
    mo = model.get_model(pp)
    psr = mo.PSR.value
    onoffs = np.sort(glob(f"data/{psr}*.onoff")) # .onoff files associated with given par
    gcoord = mo.coords_as_GAL()
    [Tsky350,Tsky820] = gsm.get_sky_temperature(gcoord, freqs=np.array([350.0,820.0])) # Includes Tcmb
    
    #FluxDict[psr] = blank_flux_dict
    
    psrDict = {'Flux350':None, 'errFlux350':None, 'Flux820':None, 'errFlux820':None, 'SI':None, 'errSI':None}
    for oo_file in onoffs:
        
        ooo = OnOff(oo_file)
        
        # Get frequency-dependent info
        if "350MHz" in oo_file:
            bw = 70 # 30 pct reduction (RFI)
            Tsys = Tsky350+23.0 # Uncertainty?
            flux_key = 'Flux350'
        elif "820MHz" in oo_file:
            bw = 175 # 12.5 pct reduction (RFI)
            Tsys = Tsky820+22.0
            flux_key = 'Flux820'
            
        flux = beta*ooo.snr*Tsys*np.sqrt(ooo.duty/(1-ooo.duty))/(Gain*np.sqrt(Npol*ooo.tobs*bw))/ooo.deg
        
        # Main contributors to uncertainty = Tsys (5 K), degradation (factor due to offset, 5-30%), bw (10 MHz?)
        deg_err = (1.0-ooo.deg)/2.0
        flux_err = flux*np.sqrt((deg_err/ooo.deg)**2+(5.0/Tsys)**2+0.5*(10.0/bw)**2)
        psrDict[flux_key] = flux
        psrDict[f"err{flux_key}"] = flux_err
        
        #print(f"{oo_file}: {flux:.3f}+/-{flux_err:.3f} mJy")
        #print(flux_key)
    
    # Calculate spectral indices (and error!)
    if psrDict['Flux350'] and psrDict['Flux820']:
        si = get_si(350.0,psrDict['Flux350'],820.0,psrDict['Flux820'])
        psrDict['SI'] = si
        a = np.abs(1.0/(np.log10(820.0/350)*np.log(10.0)))
        si_err = a*np.sqrt((psrDict['errFlux350']/psrDict['Flux350'])**2+(psrDict['errFlux820']/psrDict['Flux820'])**2)
        psrDict['errSI'] = si_err
    else:
        print(f"{psr} does not have enough info to calculate spectral index.")
        
    if si_err > np.abs(si):
        print(f"Spectral index is not well constrained for {psr}.")
        
    FluxDict[psr] = psrDict

In [ ]:
import table_utils as tu

# WRITE TABLE LINES AND FILE WITH FLUX INFO FOR PLOTTING PROFILES
for psr in FluxDict.keys():
    pD = FluxDict[psr]
    S350_tex = tu.ufve(pD['Flux350'],pD['errFlux350'],digits=2)
    if pD['Flux820']:
        S820_tex = tu.ufve(pD['Flux820'],pD['errFlux820'],digits=2)
        SI_tex = tu.ufve(pD['SI'],pD['errSI'],digits=2)
    else:
        S820_tex = "--"
        SI_tex = "--"
        
    print(f"{psr} {S350_tex} {S820_tex} {SI_tex}")